## ANN for question matching

- Measure Levenshtein distance $\checkmark$

- Measure intersection of words: normalized $\checkmark$

- Define type of words $\checkmark$

- Measure intersection of nouns: normalized by label nouns, then check if each is in the other synset. Count the number of shares.

- Find in each phrase the first noun or name.

- How similar the words tags are $\checkmark$

- Measure the question type: normalized

- What, if we measure the entropy and information of each sentence and compare them ?


In [1]:
import pandas as pd
import Levenshtein
import numpy as np
import nltk
#nltk.download()
from nltk.corpus import wordnet

In [2]:
test=pd.read_csv('test.csv',nrows=200)
train=pd.read_csv('train.csv',nrows=200)
sample_submission=pd.read_csv('sample_submission.csv',nrows=200)
train.question1=train.question1.str.replace('?','').str.upper()
train.question2=train.question2.str.replace('?','').str.upper()

In [23]:
def words_intersection(x,y):
    shorter=min([len(x),len(y)])
    common=set(x.split(' ')).intersection(y.split(' '))
    return len(common)/float(shorter)

def to_words_tags(s):
    text=nltk.word_tokenize(s)
    return ' '.join([x[1] for x in nltk.pos_tag(text)])
def tags_distance(q1,q2):
    q1 = to_words_tags(q1)
    q2 = to_words_tags(q2)
    return Levenshtein.ratio(q1,q2)

train['intersection'] = train.apply(lambda x: words_intersection(x['question1'],x['question2']), axis=1) # normalized intersection
train['similarity'] = train.apply(lambda x: Levenshtein.ratio(x['question1'],x['question2']), axis=1) # normalized Levenshtein
train['tag_similarity'] = train.apply(lambda x: tags_distance(x['question1'],x['question2']), axis=1) # Levenshtein distance of tagged words 

In [40]:
s = 'Tesla went on to try and develop a series of inventions'
text=nltk.word_tokenize(s)
[word for word, tag in nltk.pos_tag(text) if 'NN' in tag]

['Tesla', 'series', 'inventions']

In [42]:
# great good
for i in wordnet.synsets('series'):
    print (i)
#wordnet.synset('car.n.01').lemma_names()

Synset('series.n.01')
Synset('serial.n.01')
Synset('series.n.03')
Synset('series.n.04')
Synset('series.n.05')
Synset('series.n.06')
Synset('series.n.07')


In [16]:
from sklearn.neural_network import MLPClassifier

In [17]:
X = [[0., 0.], [1., 1.]]
y = [0, 1]

In [19]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [29]:
clf.predict([1,0])

/home/lenovo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

In [34]:
clf.predict_proba([1,0])

/home/lenovo/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[  1.00000000e+00,   1.56634271e-26]])